In [ ]:
import pandas as pd
from rdkit import Chem
import numpy as np
from tqdm import tqdm
import pickle

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole #Needed to show molecules
from rdkit.Chem.Draw.MolDrawing import MolDrawing, DrawingOptions
from mordred import Calculator, descriptors
from tqdm import tqdm
from descriptor_utils import create_momentf
import mordred
import pickle
from xyz import create_xyz
import datetime


In [ ]:
def get_xyz(path):
    f = open(path, 'r')
    f = f.readlines()
    atoms = f[2:]
    
    natoms = int(f[0].strip('\n'))
    
    
    atom_z = [atoms[i].split()[0] for i in range(natoms)]
    pos = [[float(atoms[i].split()[1:][j]) for j in range(3)] for i in range(natoms)]
#     pos = [[float(atoms[i].split()[1:][j]) for j in range(3)] for i in range(natoms) if atom_z[i] !='H']
#     atom_z = [atom_z[i] for i in range(natoms) if atom_z[i] !='H']
    
    return atom_z, np.array(pos)

def check_for_nan_coords(df_3d, xyz_dir):
    
    nans =[]
    for i in tqdm(range(df_3d.shape[0])):

        fname = df_3d.loc[i, 'cas']
    #         fname = str(fname)
        path = xyz_dir + fname + ".xyz"
    #         print(path)
        try:
            anum, pos = get_xyz(path)

            if len(np.where(np.isnan(pos.ravel()))[0]) !=0:
                nans.append(fname)

        except:
            nans.append(fname)
    #     break
    return nans

def find_missing(df_3d, features):
    
    missing_features = []
    for f in features:    
        for i in df_3d.loc[:, f].values:
            if isinstance(i , mordred.error.Missing):
                missing_features.append(f)
    #             print(i)
                break
    return missing_features

def get_float_features(dft):
    
    floats=[]
    for i in dft.columns:
        try:
            dft[i].astype(float)
            floats.append(i)
        except:
            pass
    return floats

# location of the xyz coordinate

print("code started")
xyz_dir = 'your_path/pybel_xyz/with_H/'

with open("your_path/fragments.dat", "rb") as f:
    fragments = pickle.load(f)

# load the data
df = pd.read_csv("your_path/data.csv", encoding='iso-8859-1')

# convert smiles to canonical format
df['rdkit_smiles'] = np.nan
for i in tqdm(range(df.shape[0])):
    try:
        df.loc[i,'rdkit_smiles'] = Chem.MolToSmiles(Chem.MolFromSmiles(df.loc[i, 'SMILES']))
    except:
        pass
    

df['log_sol'] = np.log10(1e-3*df['Experimental Solubility in Water']/df['Molar Mass'])
df.reset_index(drop=True, inplace=True)
dups = df[df.duplicated(subset=['rdkit_smiles'])]

if dups.shape[0] !=0:
    print("Warning: There are duplicate smiles!")

else:    
    # create pybel xyz data
    create_xyz(df)

    # calculate mordred features
    calc = Calculator(descriptors, ignore_3D=True)
    mols = [Chem.MolFromSmiles(i) for i in df.rdkit_smiles.values]
    df_3d = calc.pandas(mols)
    features = df_3d.columns.tolist()

    # find features containing missing values
    # such columns are removed
    missing_features = find_missing(df_3d, features)      
    newf = list(set(df_3d.columns.values).difference(list(set(missing_features))))
    df_3d = df_3d.loc[:, newf]
    
    # only use features that can be converted to floats
    floats = get_float_features(df_3d)
    df_3d = df_3d.loc[:, floats]
    
    # drop other nan columns
    df_3d = df_3d.astype(float)
    df_3d.dropna('columns', inplace=True)
    df_3d.reset_index(drop=True, inplace=True)

    # add the important data
    df_3d.loc[:, 'smiles'] = df.rdkit_smiles
    df_3d.loc[:, 'log_sol'] = df.log_sol
    df_3d.loc[:, 'cas'] = df['CAS Number']
    df_3d.loc[:, 'ref'] = df['Experiment Reference']
    df_3d.loc[:, 'temp'] = df['Temperature']
    df_3d.loc[:, 'inchi'] = df['Standard InChIKey']

    df_3d = create_momentf(df_3d, xyz_dir, 'cas', fragments)


    # check whether there are nan coordinates or xyz coordinates are unavailable
    nans = check_for_nan_coords(df_3d, xyz_dir)
    df_3d = df_3d[~df_3d.cas.isin(nans)]
    df_3d.reset_index(drop=True, inplace=True)


    # split the data into train/valid/test sets
    df_3d.loc[:,"sol_id"] = np.nan
    df_3d.loc[df_3d.log_sol<=-8,"sol_id"] = 0
    df_3d.loc[(df_3d.log_sol>-8) & (df_3d.log_sol<=-6),"sol_id"] = 1
    df_3d.loc[(df_3d.log_sol>-6) & (df_3d.log_sol<=-4),"sol_id"] = 2
    df_3d.loc[(df_3d.log_sol>-4) & (df_3d.log_sol<=-2),"sol_id"] = 3
    df_3d.loc[(df_3d.log_sol>-2) & (df_3d.log_sol<=0),"sol_id"] = 4
    df_3d.loc[df_3d.log_sol>0 ,"sol_id"] = 5

    train, test = train_test_split(df_3d, stratify = df_3d.sol_id.values, test_size=0.15)
    val, test = train_test_split(test, stratify = test.sol_id.values, test_size=0.5)

    train.drop(['sol_id'], axis=1, inplace=True)
    test.drop(['sol_id'], axis=1, inplace=True)
    val.drop(['sol_id'], axis=1, inplace=True)

    train.to_csv("your_path/train.csv", index=False)
    test.to_csv("your_path/test.csv", index=False)
    val.to_csv("your_path/val.csv", index=False)

    print(f"data creation completed at {datetime.datetime.now()}")


code started.


  5%|▌         | 591/11696 [00:14<03:03, 60.48it/s] 

c:\Users\haozh\.conda\envs\deepchem\Lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
c:\Users\haozh\.conda\envs\deepchem\Lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


  7%|▋         | 856/11696 [00:17<01:48, 100.25it/s]

c:\Users\haozh\.conda\envs\deepchem\Lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


...

c:\Users\haozh\.conda\envs\deepchem\Lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


 96%|█████████▌| 11212/11696 [02:50<00:07, 66.43it/s]

c:\Users\haozh\.conda\envs\deepchem\Lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


100%|██████████| 11696/11696 [02:59<00:00, 65.33it/s] 
C:\Users\haozh\AppData\Local\Temp\ipykernel_34700\1007700888.py:108: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only.
  df_3d.dropna('columns', inplace=True)
100%|██████████| 11696/11696 [00:01<00:00, 6353.90it/s]


data creation completed at 2025-11-09 21:42:47.039788
